In [183]:
import numpy as np
import random 
from collections import Counter
from statistics import mean, median
import matplotlib.pyplot as plt
%matplotlib inline

# Gym import
import gym


# keras import
from keras.models import Sequential
from keras.layers import Dense, Flatten ,Activation 
from keras.models import load_model
from keras.layers.convolutional import Conv1D, MaxPooling1D


In [43]:
Dataset = np.load('data.npy')
len(Dataset)


def process_data(data):
    cropped = data[34:-16,:]
    if len(cropped.shape) == 3:
        return np.mean(cropped[::2,::2,:], axis=2)
    elif len(cropped.shape) == 4:
        downsample = np.mean(cropped[:, ::2, ::2], axis=3)
        return downsample.reshape(*downsample.shape, 1)
    else:
        raise ValueError()
        
def show(img):
    if not hasattr(img, 'shape'): 
        raise ValueError
    
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img)  
        

In [111]:
model = Sequential()
model.add(Conv2D(210, (3, 3), input_shape=(210, 160, 3), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
#model.add(Dense(100, activation="relu", kernel_initializer="uniform"))
model.add(Dense(1, activation="sigmoid", kernel_initializer="uniform"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 210, 160, 210)     5880      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 105, 80, 210)      0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 1764000)           0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 1764001   
Total params: 1,769,881
Trainable params: 1,769,881
Non-trainable params: 0
_________________________________________________________________


In [178]:
model = Sequential()
model.add(Conv1D(256, 80, input_shape=(80,80), padding="same"))
model.add(MaxPooling1D(pool_size=80))
model.add(Flatten())
model.add(Dense(100, activation="relu", kernel_initializer="uniform"))
model.add(Dense(1, activation="softmax ", kernel_initializer="uniform"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

ValueError: Unknown activation function:softmax 

In [187]:
model = Sequential()

model.add(MaxPooling1D(pool_size=80))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))

# Add output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [188]:
X_train = np.array([process_data(i[0]) for i in Dataset])

f = lambda x:x if x == 1 else 0
y_train = np.array([f(i[1]) for i in Dataset])


In [189]:
X_train[0].shape

(80, 80)

In [190]:
model.fit(X_train,y_train,epochs=5,verbose=1)

Epoch 1/5
600/600 [==============================] - 1s 2ms/step - loss: 5.5148 - acc: 0.4883
Epoch 2/5
600/600 [==============================] - 0s 307us/step - loss: 4.9679 - acc: 0.5100
Epoch 3/5
600/600 [==============================] - 0s 233us/step - loss: 4.2629 - acc: 0.5417
Epoch 4/5
600/600 [==============================] - 0s 173us/step - loss: 4.3981 - acc: 0.5350
Epoch 5/5
600/600 [==============================] - 0s 180us/step - loss: 4.7724 - acc: 0.5050


In [192]:
print(y_train[2])

1


In [199]:
model.predict(X_train[2].reshape((1, 80,80)))


array([[0.00053986]], dtype=float32)

In [194]:
env = gym.make('Pong-v0')
env.reset()

# Setting training parmaters
max_frames = 200
score_requirement = 1
inital_games = 20

def Pong_random_action():
    r = random.randrange(0,2)
    if r == 0:
        return 2 ## Up
    else:
        return 5 ## Down

    
def train():
    scores = []
    choices = []
    for each_game in range(inital_games):
        score = 0
        game_memory = []
        prev_obs = []
        env.reset()
        for _ in range(max_frames):
            env.render()
            if len(prev_obs) == 0:
                action = Pong_random_action()
            else:
                #predict based on our model
                action = model.predict(prev_obs.reshape((1, 80,80)))
            
            choices.append(action)
            observation, reward, done, info = env.step(action)
            prev_obs = process_data(observation)
            game_memory.append([observation,action])
            score += reward
            
            if done:
                break
        scores.append(score)
        
    print("\nAverage scores: ",mean(scores))
    print("Chose UP ",choices.count(2)/len(choices)," chose Down ",choices.count(5)/len(choices))


In [195]:
train()

IndexError: arrays used as indices must be of integer (or boolean) type

In [196]:
env.close()